# Fake News Classifier

Overview: Develop a machine learning program to identify when an article might be fake news.

Dataset: https://www.kaggle.com/c/fake-news/data

##### Dataset overview: 

train.csv: A full training dataset with the following attributes:

id: unique id for a news article
title: the title of a news article
author: author of the news article
text: the text of the article; could be incomplete
label: a label that marks the article as potentially unreliable
1: unreliable
0: reliable

test.csv: A testing training dataset with all the same attributes at train.csv without the label.

## Importing necessary libraries

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## Reading the data

In [4]:
train_data = pd.read_csv("train.csv")

In [5]:
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## Data Pre-Processing

In [9]:
### Checking the shape of the train data

train_data.shape

(20800, 5)

In [10]:
### let's check if there are any null values

train_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

We can see that there are 558 null values in 'title' feature, 1957 null values in 'author' feature and 39 null values in 'text' feature. Let's drop these null values as we cannot fill them here.

In [12]:
train_data = train_data.dropna()

In [25]:
### Checking if there are any null values remaining and the shape of data after dropping the null values

print('Shape of the data:\n',train_data.shape)

print('\nNull values present in features:\n',train_data.isnull().sum())

Shape of the data:
 (18285, 5)

Null values present in features:
 id        0
title     0
author    0
text      0
label     0
dtype: int64


We can see that the number of rows in the data set is reduced and there are no null values present in any of the features.

In [27]:
train_data.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


We can observe that 6th row has been removed due to presence of null values. We need to reset the index here.

In [28]:
train=train_data.copy()

In [29]:
train.reset_index(inplace=True)

In [30]:
train.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [31]:
### Let's split the data into independent and dependent features.

x = train.drop('label', axis=1)
y = train['label']

In [33]:
### Here we are importing nltk, stopwords and porterstemmer as we are using stemming on the text data. 
### We use stopwords to remove the stopwords present in the text data.

### re is regular expressions used for identifying only words in the text and ignoring anything else

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

### importing tensorflow and checking its version

import tensorflow as tf
tf.__version__

'2.0.0'

In [38]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(train)):
    heading = re.sub('[^a-zA-Z]', ' ', train['title'][i])
    heading = heading.lower()
    heading = heading.split()
    
    heading = [ps.stem(word) for word in heading if not word in stopwords.words('english')]
    heading = ' '.join(heading)
    corpus.append(heading)

In [39]:
corpus[2]

'truth might get fire'

In [40]:
# Setting vocabulary size
voc_size=5000

In [41]:
### Performing one hot representation

from tensorflow.keras.preprocessing.text import one_hot
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3950, 3694, 1530, 4665, 606, 2984, 1204, 3295, 3912, 1729],
 [4626, 336, 936, 4705, 4222, 4741, 4973],
 [1762, 1153, 4839, 344],
 [3132, 2400, 3217, 2724, 1395, 4318],
 [790, 4222, 3056, 60, 4893, 2065, 4222, 3461, 110, 639],
 [3246,
  536,
  4771,
  918,
  2164,
  2381,
  237,
  1794,
  4336,
  3421,
  4413,
  2059,
  1377,
  4167,
  4973],
 [1834, 1946, 1879, 3634, 2994, 1973, 1673, 1177, 1043, 3344, 2336],
 [2415, 4037, 4711, 2004, 3836, 930, 2381, 3134, 1043, 3344, 2336],
 [652, 2098, 342, 2473, 850, 283, 4845, 4107, 2381, 4457],
 [4374, 3142, 2203, 4992, 3243, 3193, 4401, 2098],
 [1800, 2603, 3569, 1852, 752, 2425, 589, 1173, 4619, 3302, 4506],
 [2724, 3595, 606, 283, 2381, 3836],
 [3822, 3105, 2629, 607, 3735, 3106, 4118, 4058, 2206],
 [846, 3565, 1987, 238, 4660, 2772, 2103, 1043, 3344, 2336],
 [4591, 3790, 4879, 1570, 3901, 1043, 3344, 2336],
 [2135, 2135, 213, 4476, 1165, 3385, 2975, 904, 869, 1177],
 [4339, 3595, 336],
 [1598, 2403, 2904, 1360, 2381, 3796, 4461, 4973],
 [26

In [44]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
### Specifying a sentence length so that every sentence in the corpus will be of same length

sentence_length = 25

### Padding to create equal length sentences


embedded_docs = pad_sequences(onehot_repr, padding='pre',maxlen=sentence_length)
print(embedded_docs)

[[   0    0    0 ... 3295 3912 1729]
 [   0    0    0 ... 4222 4741 4973]
 [   0    0    0 ... 1153 4839  344]
 ...
 [   0    0    0 ... 1043 3344 2336]
 [   0    0    0 ... 4953 2405 1269]
 [   0    0    0 ... 3382 2302 1136]]


## Building Models

In [47]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [49]:
#Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sentence_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 25, 40)            200000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 25, 40)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [50]:
z =np.array(embedded_docs)
y =np.array(y)

In [52]:
### Printing the shape 
z.shape, y.shape

((18285, 25), (18285,))

## Splitting and Training

In [99]:
### Splitting the data for training and testing the model

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(z, y, test_size=0.10)

## Model Training

In [100]:
### Finally Training
model.fit(x_train,y_train,validation_data = (x_test, y_test), epochs = 10, batch_size = 64)

Train on 16456 samples, validate on 1829 samples
Epoch 1/10
16456/16456 [==============================] - 20s 1ms/sample - loss: 0.1122 - accuracy: 0.9637 - val_loss: 0.1148 - val_accuracy: 0.9574
Epoch 2/10
16456/16456 [==============================] - 21s 1ms/sample - loss: 0.0680 - accuracy: 0.9777 - val_loss: 0.1271 - val_accuracy: 0.9557
Epoch 3/10
16456/16456 [==============================] - 20s 1ms/sample - loss: 0.0502 - accuracy: 0.9835 - val_loss: 0.1487 - val_accuracy: 0.9513
Epoch 4/10
16456/16456 [==============================] - 20s 1ms/sample - loss: 0.0404 - accuracy: 0.9866 - val_loss: 0.1777 - val_accuracy: 0.9475
Epoch 5/10
16456/16456 [==============================] - 20s 1ms/sample - loss: 0.0307 - accuracy: 0.9905 - val_loss: 0.1962 - val_accuracy: 0.9464
Epoch 6/10
16456/16456 [==============================] - 20s 1ms/sample - loss: 0.0237 - accuracy: 0.9930 - val_loss: 0.2420 - val_accuracy: 0.9410
Epoch 7/10
16456/16456 [==============================] -

In [129]:
### Loading our test dataset for prediction

test=pd.read_csv('test.csv') 
test_id=test["id"]

In [130]:
test_id

0       20800
1       20801
2       20802
3       20803
4       20804
        ...  
5195    25995
5196    25996
5197    25997
5198    25998
5199    25999
Name: id, Length: 5200, dtype: int64

In [131]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [132]:
### Removing columns that are not so important
test=test.drop(['text','id','author'], axis=1)

In [133]:
test.head()

,title
0,"Specter of Trump Loosens Tongues, if Not Purse..."
1,Russian warships ready to strike terrorists ne...
2,#NoDAPL: Native American Leaders Vow to Stay A...
3,"Tim Tebow Will Attempt Another Comeback, This ..."
4,Keiser Report: Meme Wars (E995)


In [134]:
test.shape

(5200, 1)

In [135]:
test.isnull().sum()

title    122
dtype: int64

In [136]:
test.

In [137]:
test.shape

(5200, 1)

In [138]:
### Creating corpus for the test dataset exactly the same as we created for the training dataset

ps = PorterStemmer()
corpus_test = []
for i in range(0, len(test)):
    heading = re.sub('[^a-zA-Z]', ' ', test['title'][i])
    heading = heading.lower()
    heading = heading.split()
    
    heading = [ps.stem(word) for word in heading if not word in stopwords.words('english')]
    heading = ' '.join(heading)
    corpus_test.append(heading)

In [139]:
### Creating one hot representation for the test corpus

one_hot_rep_test=[one_hot(words,voc_size)for words in corpus_test] 

In [140]:
### Padding for the test dataset
sentence_length=25

embedded_docs_test=pad_sequences(one_hot_rep_test, padding='pre', maxlen=sentence_length)
print(embedded_docs_test)

[[   0    0    0 ... 1043 3344 2336]
 [   0    0    0 ... 4906 4643 2919]
 [   0    0    0 ... 1513 1297  400]
 ...
 [   0    0    0 ... 1043 3344 2336]
 [   0    0    0 ... 1800  119 4439]
 [   0    0    0 ... 1043 3344 2336]]


In [141]:
x_test = np.array(embedded_docs_test)

In [142]:
### Making predictions for the test dataset
y_pred = model.predict_classes(x_test)

## Performance Metrics and Accuracy

#### The performance and accuracy can be checked using confusion_matrics. Here its not possible as the size of 'y_test' and 'y_pred' are different.

## Generating the result file

In [149]:
val=[]
for i in y_pred:
    val.append(i[0])

In [152]:
result_file = pd.DataFrame({'id':test_id, 'label':val})
result_file.shape

(5200, 2)

In [154]:
result_file.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,0
4,20804,1
